# N Back Task
## Optimal Behaviour

#### Deterministic Task

The task is deterministic thus programming the best possible behaviour is trivial:
- Store the sequence of information
- Trigger if matches


--------

# Formalisation

Here we provide a mathematical formalism to represent the _N-back_ task. The _optimal agent_ will assume the known correct probabilities, whilst the data can will be used to infer parameters that capture these probabilities - allowing us to measure the deviation from optimal performance across individuals.


-------

## State Space

The _N Back_ task can be considered a dichotomous state space, with two mutually exclusive states representing whether or not the current stimuli (lekker) matches that of the stimuli _N_ steps prior. That is:


$$State: S = \{Y, N\}$$

Where: 

$$
\begin{equation}
 Y = \text{Yes: there is a match} \\
 N = \text{No: there is not a match} 
\end{equation}
$$

Where a particular instance is denoted by the lowercase $y$ or $n$ respectively. Further, there is no desire to learn transition dynamics as the states are entirely independent.

$$\text{The function of the state space is only to constrain the action space, by conditioning the action space on the state space.}$$


-----------
## Action Space

Each participant has independent parameters quantifying the probability of the possible actions. (Note: the individual parameters can later incorporate joint distributional information by incorporating Bayesian priors over all participants).

The action space is dichotomous & defined as:


- *C* - _*Correct*_:      accurately signaling whether or not there is a match.
- *I* - _*Incorrect*_:    inaccurately signaling a match erroneously


Thus:

$$Action \ Space: \{C, I\}$$


We can represent the likelikhood of taking a specific action as non-stationary probabilities (non-stationary as they change over the course of the task, thus indexed by time $t$). An individual probability should be learnt for each participant $p$:


\begin{equation}
\delta_t^p(j) \ \ for \ \  j=\{c, i\}
\end{equation}

That is:


\begin{equation}
\delta_t^p(c) = \text{probability of partipicpant $p$ takes the correct action at time $t$}\\
\delta_t^p(i) = \text{probability of partipicpant $p$ takes the incorrect action at time $t$}
\end{equation}


#### Dichotomous Actions

Because the actions are binary, we can simply assume:

$$\delta_t^p(c) = 1 - \delta_t^p(i)$$

Thus only need to model:

$$\delta_t^p(c)$$



#### Dependence on State

The action space is conditioned on the state $s$. Inituitively, one can expect the probability of taking the correct action to be higher when the correct action is to do nothing then when the correct action is to signal a match. 

Therefore the parameters ought to capture this dependence on state:

\begin{equation}
\delta_t^p(c|s) \ \ \ \ where \ s=:\{y,n\}
\end{equation}

Thus we arrive at:

\begin{equation}
\delta_t^p(c|s) = \text{probability of partipicpant $p$ takes the correct action $c$ at time $t$, given state $s$}\\
\end{equation}



------ 

## Optimal model
The optimal model simply results in 

$$\delta_t^p(c|s) = 1$$

Our primary interest is to assess how different individuals deviate from this optimality by learning the probabilities as a parameter across individuals.



------

# Instantiate Model

Writing the model....

In [1]:
# !conda activate dynocog
# !conda init
# !conda install pandas -y
# !conda install -c pytorch pytorch -y
# !conda install -c conda-forge numpyro
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import torch
import sys
from tqdm import tqdm
import pickle
import plotly.express as px
import plotly.graph_objects as go

# ---- load data module ----x
import sys
sys.path.append('../')
import process_data.process_raw_data as prd
from process_data.process_raw_data import batch_processing 

In [6]:
# ---- reprocess raw data ----x
path  = '../data/data_sample/'
path2 = '../data/data_samples_pandas/'
bp    = prd.batch_processing(path)

bp.create_wcst_data()
bp.create_navon_data()
bp.create_nback_data()
bp.create_corsi_data()
bp.create_fitts_data()
bp.convert_data_to_int()
bp.write_to_pickle(path2)
bp.read_from_pickle(path2)
bp.write_class_to_pickle(path2)



        ------------------------------------------------------------------
                                WCST data created
        ------------------------------------------------------------------

        


        ------------------------------------------------------------------
                                Navon data created
        ------------------------------------------------------------------

        


        ------------------------------------------------------------------
                                N back data created
        ------------------------------------------------------------------

        


        ------------------------------------------------------------------
                                Corsi data created
        ------------------------------------------------------------------

        


        ------------------------------------------------------------------
                                Fitts data created
        ------------

In [7]:
# ---- fetch data object ----x
with open('../data/data_samples_pandas/batch_processing_object.pkl', 'rb') as file2:
    bp = pickle.load(file2)
bp.__dict__.keys()

dict_keys(['path', 'mapping', 'data_times', 'participants', 'parti_code', 'n', 'wcst_paths', 'nback_paths', 'corsi_paths', 'fitts_paths', 'navon_paths', 'wcst_data', 'nback_data', 'corsi_data', 'fitts_data', 'navon_data'])

In [15]:
sub = bp.nback_data.loc[bp.nback_data.participant ==851366.0, ]

In [17]:
sub.head()

,participant,participant_code,block_number,score,status,miss,false_alarm,reaction_time_ms,match,stimuli,stimuli_n_1,stimuli_n_2
0,851366.0,s.32ff642a-efe0-436f-8075-fa703d677fed.txt,1,1,0,1,0,0,0,3000,1,14
1,851366.0,s.32ff642a-efe0-436f-8075-fa703d677fed.txt,1,2,0,1,0,0,0,3000,2,13
2,851366.0,s.32ff642a-efe0-436f-8075-fa703d677fed.txt,1,3,0,1,0,0,0,3000,3,10
3,851366.0,s.32ff642a-efe0-436f-8075-fa703d677fed.txt,1,4,1,0,0,1,0,3000,1,13
4,851366.0,s.32ff642a-efe0-436f-8075-fa703d677fed.txt,1,5,0,0,0,0,1,867,2,12
